In [1]:
from dotenv import load_dotenv
from azure.core.credentials import AzureKeyCredential
import os

load_dotenv()


AISEARCH_API_VERSION = os.getenv("COGSEARCH-API-VERSION")
AISEARCH_ENDPOINT = os.getenv("COGSEARCH-ENDPOINT")
AISEARCH_API_KEY = os.getenv("COGSEARCH-API-KEY")
AISEARCH_INDEX = os.getenv("COGSEARCH-INDEX")
AZURE_OPENAI_KEY = os.getenv("AZURE_OPENAI-KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI-ENDPOINT")
AZURE_OPENAI_RESOURCE_NAME = os.getenv("AZURE_OPENAI-RESOURCE-NAME")
EMBEDDING_RESOURCE = os.getenv("EMBEDDING-RESOURCE")

credential = AzureKeyCredential(AISEARCH_API_KEY) 


In [2]:
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import *




index = SearchIndex(
    name = AISEARCH_INDEX,
    fields=[
        SimpleField(name="id", type=SearchFieldDataType.String, key=True),
        SimpleField(name="category", type=SearchFieldDataType.String, filterable=True),
        SearchField(name="myvector", 
                    type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                    searchable=True,
                    vector_search_dimensions=3,
                    vector_search_profile_name="vprofile")
    ],
    vector_search=VectorSearch(
        algorithms=[HnswAlgorithmConfiguration(name="algo",
                                               parameters=HnswParameters(metric="cosine"))],
        profiles=[VectorSearchProfile(name="vprofile", algorithm_configuration_name="algo")]
    )
)

index_client = SearchIndexClient(AISEARCH_ENDPOINT, credential=credential)
index_client.create_or_update_index(index)


In [18]:
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorFilterMode, VectorizedQuery

search_client = SearchClient(AISEARCH_ENDPOINT, AISEARCH_INDEX, credential)



Add a Few Documents

In [ ]:
search_client.upload_documents(documents=[
    {"id":  "1",    "myvector": [1, 2, 3],    "category": "A" },
    {"id":  "2",    "myvector": [1, 1, 3],    "category": "A" },
    {"id":  "3",    "myvector": [4, 5, 6],    "category": "B" }

])

In [ ]:
q = [1, 3, 2]




r = search_client.search(None, vector_queries=[VectorizedQuery(vector=q, k=2, fields="myvector")])

for doc in r:
    print(f"id: {doc['id']}, score: {doc['@search.score']}, cat: {doc['category']}")